# Links
- https://www.kaggle.com/c/otto-group-product-classification-challenge

# Solutions
- 4-th place: https://github.com/diefimov/otto_2015
- 12-th place: https://github.com/tks0123456789/kaggle-Otto
- 66-th place: https://github.com/ahara/kaggle_otto

In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2

import pipeline as pl

In [ ]:
pl.test_validate()

In [19]:
stats = pd.read_json('results.json').sort_values('lloss-mean')
stats.T[stats.nunique() > 1]

,0,1
calibration,False,True
exec-time-sec,772.512,3183.97
llos-std,0.00535488,0.00323549
lloss-mean,-0.482933,-0.473761
